# 1. 사전 세팅

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%cd "/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/8_creditcard_fraud_detection"

# 2. 문제 정의

---

    대부분의 국가에서 신용카드 사용이 일반화되고 있다.
    신용카드 사용자들은 항상 분실에 대한 우려를 가지고 있고, 신용카드 회사들은 분실했을 경우,
    사용하는 패턴이 달라지는 것을 감지해 신용카드를 정지하는 시스템(Fraud Detection System)을 보유하고 있다.
    데이터를 통해 이런 패턴을 자동으로 감지할 수 있다면, 카드 오용으로 인한 피해를 미연에 방지할 수 있다.


## 2.1. 도메인 지식

---

▸ Fraud Detection System(FDS)

    현재는 범용적인 특정 조건들에 의해서 탐지를 진행하고 있다.
    예전에는 Association Rule처럼 연관되어 있는 규칙만을 따랐다면, 요즘에는 패턴 및 가족 구성원 같은 데이터도 고려하고 있다.

▸ 가게 규모에 따른 수수료율

    흔히 영세 상인과 같은 규모가 작은 곳은 수수료가 없는 편이고, 대형 마트같은 곳은 상대적으로 수수료율을 매기는 등, 수수료율을 차등으로 매긴다.

# 3. 데이터 간단 확인

## 3.1. 데이터 살펴보기

In [ ]:
import pandas as pd

df = pd.read_csv('Data/data.csv')

print(['{}:{}'.format(i, name) for (i, name) in enumerate(list(df.columns))])



```
['0:class', '1:v1', '2:v2', '3:v3', '4:v4', '5:v5', '6:v6', '7:v7', '8:v8', '9:v9', '10:v10', '11:v11', '12:v12',
 '13:v13', '14:v14', '15:v15', '16:v16', '17:v17', '18:v18', '19:v19', '20:v20', '21:v21', '22:v22', '23:v23',
 '24:v24', '25:v25', '26:v26', '27:v27', '28:v28', '29:log_amount']
```



    민감 정보이기 때문에 비식별화되어있는 컬럼 형태로 주어져있다.

In [ ]:
# 본 데이터는 feature에 대한 Description이 없기에, Feature Selection을 하기 어려움
df.columns.tolist()

In [ ]:
df.isnull().sum()

In [ ]:
print(df['class'].value_counts())



```
class
0    284315
1       492
Name: count, dtype: int64
```



## 3.2. 데이터 명세

---

| Column | Description |
| :------------------ | :------------------------- |
| Features | 신용카드 거래 관련 정보 |
| Class        | 이상거래 여부 |

In [ ]:
df.head()

In [ ]:
df.shape



```
(284807, 30)
```



## 3.3. 데이터 가공 명세

---

| 명세 | 내용 |
|:----|:----|
| 가공 범위 | <input type="checkbox"> Feature(Column) 변형<br> <input type="checkbox">기간 단위 데이터 분석 및 Big Volume 특성을 반영하기 위한 데이터 증강<br> <input type="checkbox" checked>다양한 Table을 참조하기 위한 데이터 분할|
| 가공 상세 | <b>데이터 분할</b><br><br>- Data Imbalance 특성을 고려하기 위해 5% 수준으로 under-sampling 진행<br>- over-sampling을 통해 Data Inbalance 조정 |
| 가공 결과 | <b>As is</b><br><br>- row: 284315 / column: 30 / table: 1<br><br> <b>To be</b><br><br>- row: 14216(-270099) / column: 30 / table: 1  |

# 4. 문제 해결 프로세스 정의

---

▸ 문제

    대부분의 사람들은 요즘 신용카드를 사용한다.
    신용카드는 사용에 있어서 높은 편의성을 제공하지만, 카드의 분실 혹은 도난으로 인한 이상거래가 발생할 수 있다.

    1. As-Is
    대부분의 금융 기관에서는 특정 조건(시간, 장소 등)과 개인의 특성을 매핑해 트리거를 만들어서 경보 시스템을 운영한다.
    예를 들어, 60대의 신용카드가 새벽 2시에 사용되었다면, 알림이 가는 시스템이다.

    2. To-Be
    위와 같은 조건으로 알림을 주는 것도 이상거래를 탐지할 수 있지만,
    다양한 Feature를 활용해 복합적인 상황에서의 이상거래를 탐지하기 위한 모델링을 하고자 한다.

    3. Goal
    수많은 신용카드 이용 내역 중, 이상거래를 탐지할 수 있는 모델을 만들고,
    이상거래의 경우 데이터가 매우 Imbalance하기에 오버샘플링을 진행할 예정.
    
▸ 기대 효과

    이상거래 탐지 모델링을 통한 자동 탐지 기능 적용
    이상거래를 통해 발생하는 고객의 피해 금액 감소

▸ 해결 방안

    Binary Classification을 통한 이상거래 여부 분류
    Logistic, KNN, RandomForest 이용

▸ 성과 측정

    이상 거래 탐지율

# 5. Data Readiness Check